<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

We need to create a new csv file containing unique restaurants from inspections with a bunch of useful features and computed risk score.



In [1]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")


<font color='red'>
#### We want kind of that fields
>restaurants_corpus = pd.DataFrame(columns = 
                                            
                                            ['restaurant licence',\
                                             'latitude',\
                                             'longitude',\
                                             'community area',\
                                             'restaurant name',\
                                             'rate of failed inspections',\
                                             'rate of successful inspections',\
                                             'Average nbr of important violations',\
                                             'Food chain flag', \
                                             'nbr risk_related words computed from inspectors comments',\
                                             'Risk_score'])
</font>

In [2]:
#Read the necessary cleaned datasets to keep the selected features
cfi_dataset = pd.read_pickle('datasets/cleaned_inspections.pickle')

In [5]:
cfi_dataset_selected = cfi_dataset[['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Latitude', 'Longitude', 
                                    'Location', 'Community Area', 'Inspection Date', 'Results', 'Risk', 
                                    'Violation Numbers', 'Violation Comments']].copy()

Since we examined earlier in the Milestone2, as shown below 
<img src="datasets/countresult.png" style="height:200px"> 

here we keep only the data with "Pass", "Fail", "Pass w/ Conditions" results.


In [6]:
# Keep only the wanted results from data 
cfi_dataset_selected = cfi_dataset_selected[cfi_dataset_selected['Results'].isin(['Pass', "Fail", "Pass w/ Conditions"])]

Now let's eliminate the violations with NaN values from the data because our risk score analysis will be based on violations.

In [7]:
cfi_dataset_selected = cfi_dataset_selected[~pd.isnull(cfi_dataset_selected['Violation Numbers'])]

In [8]:
cfi_dataset_selected = cfi_dataset_selected.reset_index()

In [9]:
cfi_dataset_selected = cfi_dataset_selected.drop(columns='index')

In [10]:
# Total number of violation counts
cfi_dataset_selected['Violation Counts'] = [len(cfi_dataset_selected['Violation Numbers'][i]) for i in range(len(cfi_dataset_selected))]


In [50]:
# Total number of violation counts according to some gradings 
# source: https://www1.nyc.gov/assets/doh/downloads/pdf/rii/how-we-score-grade.pdf
'''
count the violation numbers for each inspection:
if there is violation number between 0 and 13 points it will be counted as 'Critical Violations Count'
if there is violation number between 14 and 27 points it will be counted as 'Moderate Violations Count'
if there is violation number between 28 and more points it will be counted as 'Non-Critical Violations Count'
'''

cfi_dataset_selected['Critical Violations Count'] = [int(len([ i for i in cfi_dataset_selected['Violation Numbers'][k] if i<14])) for k in range(len(cfi_dataset_selected))]
cfi_dataset_selected['Moderate Violations Count'] = [int(len([ i for i in cfi_dataset_selected['Violation Numbers'][k] if i>13 and i<28])) for k in range(len(cfi_dataset_selected))]
cfi_dataset_selected['Non-Critical Violations Count'] = [int(len([ i for i in cfi_dataset_selected['Violation Numbers'][k] if i>27])) for k in range(len(cfi_dataset_selected))]


In [22]:
# Adding Allergen flag to the dataset
cfi_dataset_selected['Allergen Flag'] = np.nan
for i in range(len(cfi_dataset_selected)):
    each_comment[i] = pd.Series([x.strip() for x in cfi_dataset_selected['Violation Comments'][i]])
    if (each_comment[i].astype(str).str.contains('ALLERGEN', flags=re.IGNORECASE, regex=True)== True).any():
        cfi_dataset_selected['Allergen Flag'][i] = 'Y'
    else: 
        cfi_dataset_selected['Allergen Flag'][i] = 'N'

In [23]:
# Adding Vomit or Diarrheal Events Flag to the dataset
cfi_dataset_selected['Vomit or Diarrheal Flag'] = np.nan
for i in range(len(cfi_dataset_selected)):
    each_comment[i] = pd.Series([x.strip() for x in cfi_dataset_selected['Violation Comments'][i]])
    if (each_comment[i].astype(str).str.contains('VOMIT', flags=re.IGNORECASE, regex=True)== True).any():
        cfi_dataset_selected['Vomit or Diarrheal Flag'][i] = 'Y'
    elif (each_comment[i].astype(str).str.contains('DIARRHEAL', flags=re.IGNORECASE, regex=True)== True).any():
        cfi_dataset_selected['Vomit or Diarrheal Flag'][i] = 'Y'
    else:
        cfi_dataset_selected['Vomit or Diarrheal Flag'][i] = 'N'

In [52]:
cfi_dataset_selected.head()

,Inspection ID,DBA Name,AKA Name,License #,Latitude,Longitude,Location,Community Area,Inspection Date,Results,Risk,Violation Numbers,Violation Comments,Violation Counts,Allergen Flag,Vomit or Diarrheal Flag,Critical Violations Count,Moderate Violations Count,Non-Critical Violations Count
0,2345318,SUBWAY,SUBWAY,2529116,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2019-11-08,Pass w/ Conditions,1,"[3, 5, 58]",[ 2-102.14(O) OBSERVED NO WRITTEN EMPLOYEE HE...,3,Y,Y,2,0,1
1,2321166,SUBWAY,SUBWAY,2529116,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2019-11-06,Fail,1,"[3, 5, 10, 22, 33, 57, 58]",[ OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY O...,7,Y,Y,3,1,3
2,2290717,PAPA JOHN'S PIZZA,PAPA JOHN'S PIZZA,1545897,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2019-05-30,Pass w/ Conditions,2,"[3, 5]",[ OBSERVED NO EMPLOYEE HEALTH POLICY ON THE PR...,2,N,Y,2,0,0
3,2144588,SUBWAY,SUBWAY,2529116,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2018-02-06,Fail,1,"[16, 34, 42]",[ OBSERVED BLACK AND PINK MOLD LIKE SUBSTANCES...,3,N,N,0,1,2
4,2064444,PAPA JOHN'S PIZZA,PAPA JOHN'S PIZZA,1545897,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2017-06-16,Pass,2,"[32, 33, 34, 35, 37, 43, 45]","[ Inspector Comments, Inspector Comments, ...",7,N,N,0,0,7


In [53]:
#cfi_dataset_selected.to_pickle('pickles/cfi_dataset_selected')

In [2]:
cfi_dataset_selected_pickle = pd.read_pickle('pickles/cfi_dataset_selected')

- We can drop 'Violation Comments' column, because we do not need it anymore.

In [3]:
cfi_dataset_selected_pickle = cfi_dataset_selected_pickle.drop(['Violation Comments' ] ,axis=1)

In [4]:
cfi_dataset_selected_pickle.describe()

,Inspection ID,License #,Latitude,Longitude,Risk,Violation Counts,Critical Violations Count,Moderate Violations Count,Non-Critical Violations Count
count,1.182300e+05,1.182300e+05,118230.000000,118230.000000,118230.000000,118230.000000,118230.000000,118230.000000,118230.000000
mean,1.455131e+06,1.595582e+06,41.883736,-87.676102,1.295957,4.414971,0.419893,0.379421,3.615656
std,6.388562e+05,9.020764e+05,0.079277,0.059985,0.560864,2.910267,0.864290,0.699434,2.284933
min,4.424700e+04,0.000000e+00,41.644670,-87.914428,1.000000,1.000000,0.000000,0.000000,0.000000
25%,1.150865e+06,1.196827e+06,41.844671,-87.705859,1.000000,2.000000,0.000000,0.000000,2.000000
50%,1.497306e+06,1.979467e+06,41.892249,-87.664223,1.000000,4.000000,0.000000,0.000000,3.000000
75%,2.010082e+06,2.245453e+06,41.939869,-87.634293,1.000000,6.000000,0.000000,1.000000,5.000000
max,2.345339e+06,3.846407e+06,42.021064,-87.526940,3.000000,34.000000,9.000000,6.000000,23.000000


In [5]:
# Load yelp and fast food restaurants dataset and combine with CFI dataset
yelp_dataset = pd.read_pickle('business_details.pickle')
fast_foods = pd.read_csv('datasets/FastFoodRestaurants.csv')

In [6]:
# We only keep the restaurants that appear more than once
chain_counts = (fast_foods['name'].value_counts())
fast_food_chains = chain_counts[chain_counts.values>=2].index.tolist()

# To this list, we add some of the most famous chains that exist in the USA :
fast_food_chains.extend(['Starbucks', 'KRISPY KRUNCHY CHICKEN', 'Caribou coffee',\
                         "PEET'S COFFEE & TEA", 'PHILZ COFFEE', 'INTELLIGENTSIA', 'PROTEIN BAR']) 
# added newly on 08.12.2019
fast_food_chains.extend(['Starbucks Coffee', 'POPEYES FRIED CHICKEN', 'MCDONALDS RESTAURANT',\
                         "POPEYE'S FRIED CHICKEN",  'POPEYES FAMOUS FRIED CHICKEN', '#1 CHOP SUEY',\
                         "NANDO'S PERI-PERI", "NANDO'S PERI PERI", 'LITTLE CAESAR ENTERPRISES',\
                         "MCDONALD'S RESTAURANT", 'TARGET/STARBUCKS/PIZZA HUT', 'LITTLE CAESARS',\
                         'POPEYES FRIED CHICKEN REST', "STARBUCK'S COFFEE", 'BURGER KING #'])

# We remove the char '\'' for simplicity, transform everything to lowercase and remove duplicates
fast_food_chains = set([chain_name.replace("\'", "").lower() for chain_name in fast_food_chains])

In [7]:
# We create a new column for Chain flag for food chains
# We fill the column with 'N' for No' and 'Y' for 'Yes' if the establishment name exists in the fast_food_chains
boolean_foodchains = pd.DataFrame(cfi_dataset_selected_pickle['AKA Name'].str.replace("\'", "")\
                                  .str.lower().isin(fast_food_chains)).rename(columns={"AKA Name": "Chain flag temp"})

In [8]:
# Merge two data frames to get the chain flag
cfi_dataset_selected_pickle = cfi_dataset_selected_pickle.merge(boolean_foodchains, how='left',
                                                                left_index = True, right_index = True)

In [9]:
# By mapping the 'Chain flag temp' booleans values to 'Y' for 'Yes' and 'N' for "No"
temp = {True:'Y', False:'N'}
cfi_dataset_selected_pickle['Chain flag'] = cfi_dataset_selected_pickle['Chain flag temp'].map(temp)
cfi_dataset_selected_pickle = cfi_dataset_selected_pickle.drop(['Chain flag temp'], axis=1).reset_index(drop=True)


In the Milestone2 in 'DataCleaningAndExploratoryAnalysis.ipynb', we observed that over the inspection years, the data shows an increasing trend, except in the 2019 November. You can see the line graph that we had in the Milestone2. <img src="datasets/linegraph.png" style="height:400px"> 
Since by that time we gathered the data this is normal. But in order to avoid misinterpretations later in the risk score calculation, we will trim the data where the 'Inspection Date''s year and month is equal to '2019-11'. 

In [10]:
cfi_dataset_selected_pickle['Inspection Year'] =  [x.year for x in cfi_dataset_selected_pickle['Inspection Date']]
cfi_dataset_selected_pickle['Inspection Month'] =  [x.month for x in cfi_dataset_selected_pickle['Inspection Date']]
group_by_inspection_date = cfi_dataset_selected_pickle.groupby(by=['Inspection Year', 'Inspection Month']).size().reset_index(name="Count")
group_by_inspection_date

,Inspection Year,Inspection Month,Count
0,2010,1,778
1,2010,2,897
2,2010,3,936
3,2010,4,911
4,2010,5,980
...,...,...,...
114,2019,7,866
115,2019,8,936
116,2019,9,768
117,2019,10,775


In [11]:
before_length = len(cfi_dataset_selected_pickle)

In [12]:
# Create a dataframe where inspection date year is 2019 and inspection date month is 11
cfi_dataset_201911 = cfi_dataset_selected_pickle[(cfi_dataset_selected_pickle['Inspection Year'] == 2019) & (cfi_dataset_selected_pickle['Inspection Month'] == 11)]
combined = cfi_dataset_selected_pickle.append(cfi_dataset_201911)
cfi_dataset_selected_pickle = combined[~combined.index.duplicated(keep=False)].reset_index()
after_length = len(cfi_dataset_selected_pickle)
print('We successfully removed {} of the inpections data which come from the date of 2019.11.'.format(before_length-after_length))

We successfully removed 78 of the inpections data which come from the date of 2019.11.


In [13]:
# Let's drop unnecessary columns
cfi_dataset_selected_pickle = cfi_dataset_selected_pickle.drop(['DBA Name', 'Location', 'Violation Numbers', 'index', 
                                                                'Inspection Date', 'Inspection Year', 
                                                                'Inspection Month', 'Risk'], axis=1)
cfi_dataset_selected_pickle['Lowercased Name'] = cfi_dataset_selected_pickle['AKA Name'].str.lower()
cfi_dataset_selected_pickle.head()

,Inspection ID,AKA Name,License #,Latitude,Longitude,Community Area,Results,Violation Counts,Allergen Flag,Vomit or Diarrheal Flag,Critical Violations Count,Moderate Violations Count,Non-Critical Violations Count,Chain flag,Lowercased Name
0,2290717,PAPA JOHN'S PIZZA,1545897,41.927995,-87.785752,Belmont Cragin,Pass w/ Conditions,2,N,Y,2,0,0,Y,papa john's pizza
1,2144588,SUBWAY,2529116,41.927995,-87.785752,Belmont Cragin,Fail,3,N,N,0,1,2,Y,subway
2,2064444,PAPA JOHN'S PIZZA,1545897,41.927995,-87.785752,Belmont Cragin,Pass,7,N,N,0,0,7,Y,papa john's pizza
3,2059956,SUBWAY,2529116,41.927995,-87.785752,Belmont Cragin,Pass,1,N,N,0,0,1,Y,subway
4,2060035,PAPA JOHN'S PIZZA,1545897,41.927995,-87.785752,Belmont Cragin,Fail,9,N,N,0,0,9,Y,papa john's pizza


- In order to calculate the rate of the failed ('Fail' results) inspections and successful (here we consider successful as combination of 'Pass' and 'Pass w/ Conditions') inspections we will do following steps:

In [14]:
grouped_by_results = cfi_dataset_selected_pickle[['Inspection ID', 'AKA Name', 'Lowercased Name', 'Chain flag',
                                                  'License #', 'Community Area', 'Latitude', 'Longitude', 'Results']]\
                                        .groupby(by=['AKA Name', 'Lowercased Name', 'Chain flag', 'License #', 
                                                     'Community Area', 'Latitude', 'Longitude', 'Results'])\
                                        .size().reset_index(name="Count")


In [15]:
# Mapping the 'Pass w/ Conditions' to 'Pass' and keeping all other results as it is
grouped_by_results['Results'] = grouped_by_results['Results'].map({'Pass w/ Conditions': 'Pass', 
                                                                  'Pass': 'Pass',
                                                                  'Fail': 'Fail'})

In [16]:
# Get unique establishments' names, chain flag, licence number, community areas, latitude and longitude
grouped_aka_names_area = grouped_by_results[['AKA Name', 'Lowercased Name', 'Chain flag', 'License #', 
                                             'Community Area', 'Latitude', 'Longitude']].drop_duplicates()\
                                            .reset_index()


In [17]:
grouped_aka_names_area = grouped_aka_names_area.drop('index', axis=1)

In [18]:
# Define a function to get sum over all 'Pass' results and 'Fail' results separately in a two variables
def get_sum_results(df):
    df = df.reset_index()
    df = df.drop('index', axis=1)
    succes_sum = sum(df[(df['Results'] == 'Pass')]['Count'])
    fail_sum = sum(df[(df['Results'] == 'Fail')]['Count'])
    return [succes_sum, fail_sum]

In [19]:
# Create a new column named 'Successful Inspection Count' for total number of the pass counts
grouped_aka_names_area['Successful Inspection Count'] = [get_sum_results(grouped_by_results[(grouped_by_results['AKA Name']==grouped_aka_names_area['AKA Name'][i]) & (grouped_by_results['Community Area']==grouped_aka_names_area['Community Area'][i])])[0] for i in range(len(grouped_aka_names_area))]
                                                    

In [20]:
# Create a new column named 'Failed Inspection Count' for total number of the fail counts
grouped_aka_names_area['Failed Inspection Count'] = [get_sum_results(grouped_by_results[(grouped_by_results['AKA Name']==grouped_aka_names_area['AKA Name'][i]) & (grouped_by_results['Community Area']==grouped_aka_names_area['Community Area'][i])])[-1] for i in range(len(grouped_aka_names_area))]
                                                

In [21]:
# a = grouped_aka_names_area[grouped_aka_names_area['Chain flag']=='N']
# a = a[['AKA Name','Latitude', 'Longitude']].drop_duplicates().reset_index()
# a = a.drop('index', axis=1)
# b = a.groupby(by=['AKA Name']).size().reset_index(name="Count")
# b1 = b[b['Count']>1].reset_index()
# set(b1['AKA Name'])

- In order to aggregate the 'Vomit or Diarrheal Flag' feature we need to follow similar steps that we did above:

In [22]:
grouped_by_vomitDiarrheal = cfi_dataset_selected_pickle[['Inspection ID', 'AKA Name', 'Lowercased Name', 'Chain flag',
                                                         'License #', 'Community Area', 'Latitude', 'Longitude', 
                                                         'Allergen Flag', 'Vomit or Diarrheal Flag']]\
                                        .groupby(by=['AKA Name', 'Lowercased Name', 'Chain flag', 'License #', 
                                                     'Community Area', 'Latitude', 'Longitude', 
                                                     'Vomit or Diarrheal Flag'])\
                                        .size().reset_index(name="Count")


In [23]:
# Define a function to get sum over all 'Yes' and 'No' Vomit or Diarrheal Flag separately in a two variables
def get_count_VomitDiarrheal(df):
    df = df.reset_index()
    df = df.drop('index', axis=1)
    yes_count = sum(df[(df['Vomit or Diarrheal Flag'] == 'Y')]['Count'])
    no_count = sum(df[(df['Vomit or Diarrheal Flag'] == 'N')]['Count'])
    return [yes_count, no_count]

In [24]:
# Create a new column named 'VomitDiarrheal Yes Counts' for total number of the yes counts
grouped_aka_names_area['VomitDiarrheal Yes Counts'] = [get_count_VomitDiarrheal(grouped_by_vomitDiarrheal[(grouped_by_vomitDiarrheal['AKA Name']==grouped_aka_names_area['AKA Name'][i]) & (grouped_by_vomitDiarrheal['Community Area']==grouped_aka_names_area['Community Area'][i])])[0] for i in range(len(grouped_aka_names_area))]
  

In [25]:
# Create a new column named 'VomitDiarrheal No Counts' for total number of the no counts
grouped_aka_names_area['VomitDiarrheal No Counts'] = [get_count_VomitDiarrheal(grouped_by_vomitDiarrheal[(grouped_by_vomitDiarrheal['AKA Name']==grouped_aka_names_area['AKA Name'][i]) & (grouped_by_vomitDiarrheal['Community Area']==grouped_aka_names_area['Community Area'][i])])[-1] for i in range(len(grouped_aka_names_area))]
  

- In order to aggregate the 'Allergen Flag' feature we need to follow similar steps that we did above:

In [26]:
grouped_by_allergen = cfi_dataset_selected_pickle[['Inspection ID', 'AKA Name', 'Lowercased Name', 'Chain flag',
                                                  'License #', 'Community Area', 'Latitude', 'Longitude', 
                                                  'Allergen Flag', 'Vomit or Diarrheal Flag']]\
                                        .groupby(by=['AKA Name', 'Lowercased Name', 'Chain flag', 'License #', 
                                                     'Community Area', 'Latitude', 'Longitude', 'Allergen Flag'])\
                                        .size().reset_index(name="Count")


In [27]:
# Define a function to get sum over all 'Yes' and 'No' Vomit or Diarrheal Flag separately in a two variables
def get_count_allergen(df):
    df = df.reset_index()
    df = df.drop('index', axis=1)
    yes_count = sum(df[(df['Allergen Flag'] == 'Y')]['Count'])
    no_count = sum(df[(df['Allergen Flag'] == 'N')]['Count'])
    return [yes_count, no_count]

In [28]:
# Create a new column named 'Allergen Yes Counts' for total number of the yes counts
grouped_aka_names_area['Allergen Yes Counts'] = [get_count_allergen(grouped_by_allergen[(grouped_by_allergen['AKA Name']==grouped_aka_names_area['AKA Name'][i]) & (grouped_by_allergen['Community Area']==grouped_aka_names_area['Community Area'][i])])[0] for i in range(len(grouped_aka_names_area))]
  

In [29]:
# Create a new column named 'N VomitDiarrheal Counts' for total number of the no counts
grouped_aka_names_area['Allergen No Counts'] = [get_count_allergen(grouped_by_allergen[(grouped_by_allergen['AKA Name']==grouped_aka_names_area['AKA Name'][i]) & (grouped_by_allergen['Community Area']==grouped_aka_names_area['Community Area'][i])])[-1] for i in range(len(grouped_aka_names_area))]
  

- To aggregate all the violations count:

In [30]:
grouped_violations_sum = cfi_dataset_selected_pickle.groupby(by=['AKA Name', 'Lowercased Name', 
                                                                 'Chain flag', 'License #',
                                                                 'Community Area', 'Latitude', 
                                                                 'Longitude'])\
                                                    .agg({'Violation Counts': 'sum',
                                                          'Critical Violations Count': 'sum',
                                                          'Moderate Violations Count': 'sum',
                                                          'Non-Critical Violations Count': 'sum',
                                                          }).reset_index()
               

In [31]:
# Merge two datasets that we aggregated above: grouped_violations_sum and grouped_aka_names_area
features_dataframe = grouped_violations_sum.merge(grouped_aka_names_area)

In [33]:
# Add columns for total counts and rate of counts and flags to use further analysis in score calculation
features_dataframe['Total Inspections Result Counts'] = (features_dataframe['Successful Inspection Count'] + features_dataframe['Failed Inspection Count'])
features_dataframe['Total VomitDiarrheal Flag Counts'] = (features_dataframe['VomitDiarrheal Yes Counts'] + features_dataframe['VomitDiarrheal No Counts'])
features_dataframe['Total Allergen Flag Counts'] = (features_dataframe['Allergen Yes Counts'] + features_dataframe['Allergen No Counts'])

features_dataframe['Success Ratio of Inspections'] = round(100*(features_dataframe['Successful Inspection Count']/features_dataframe['Total Inspections Result Counts']),2)
features_dataframe['Failure Ratio of Inspections'] = round(100*(features_dataframe['Failed Inspection Count']/features_dataframe['Total Inspections Result Counts']),2)
features_dataframe['Yes Ratio of VomitDiarrheal']  = round(100*(features_dataframe['VomitDiarrheal Yes Counts']/features_dataframe['Total VomitDiarrheal Flag Counts']),2)
features_dataframe['No Ratio of VomitDiarrheal']   = round(100*(features_dataframe['VomitDiarrheal No Counts']/features_dataframe['Total VomitDiarrheal Flag Counts']),2)
features_dataframe['Yes Ratio of Allergen']        = round(100*(features_dataframe['Allergen Yes Counts']/features_dataframe['Total Allergen Flag Counts']),2)
features_dataframe['No Ratio of Allergen']         = round(100*(features_dataframe['Allergen No Counts']/features_dataframe['Total Allergen Flag Counts']),2)

features_dataframe['Critical Violations Ratio']     = round(100*(features_dataframe['Critical Violations Count']/features_dataframe['Violation Counts']),2)
features_dataframe['Moderate Violations Ratio']     = round(100*(features_dataframe['Moderate Violations Count']/features_dataframe['Violation Counts']),2)
features_dataframe['Non-Critical Violations Ratio'] = round(100*(features_dataframe['Non-Critical Violations Count']/features_dataframe['Violation Counts']),2)


- Yelp dataset to find out restaurants' ratings and price tags given by Yelp users:

In [40]:
# Keep only the fetaures that we need
yelp_dataset = yelp_dataset[['coordinates.latitude', 'coordinates.longitude', 'name', 'price', 'rating']]
yelp_dataset['Lowercased Name'] = yelp_dataset['name'].str.lower()
yelp_dataset = yelp_dataset.drop_duplicates()
yelp_dataset.head()

,coordinates.latitude,coordinates.longitude,name,price,rating,Lowercased Name
0,41.820629,-87.699310,Carniceria Y Fruteria Los Altos,$,3.5,carniceria y fruteria los altos
1,41.878517,-87.626351,Al's Beef,$,3.0,al's beef
2,41.968245,-87.715123,Peking Mandarin,$,4.0,peking mandarin
3,41.944680,-87.727680,La Humita,$$,4.0,la humita
4,41.968636,-87.716194,Chicago Produce,$,3.5,chicago produce


In [41]:
# Examine distinct price levels in the yelp dataset price column and map them to a numerical grade to make it easier
# further analysis on that feature
yelp_dataset['price'].unique()

array(['$', '$$', nan, '$$$', '$$$$'], dtype=object)

In [42]:
# Create a dictionary for price values:
# 0 for nan price value 
# 1 for '$' price level 
# 2 for '$$' price level 
# 3 for '$$$' price level 
# 4 for '$$$$' price level  
temp = {'$':1, '$$':2, np.nan:0, '$$$':3, '$$$$':4}
yelp_dataset['Price'] = yelp_dataset['price'].map(temp)

In [43]:
# Aggregate the Yelp dataset to have the Price, review and rating for each establishment
yelp_dataset_grouped = yelp_dataset.groupby(by=['Lowercased Name'])\
                                   .agg({'Price':'median',  
                                         'rating':'mean'}).reset_index()
yelp_dataset_grouped.head()

,Lowercased Name,Price,rating
0,!hello tacos!,2.0,3.5
1,1 chop suey,0.0,2.5
2,1000 liquors,2.0,2.5
3,101 n wacker dr,0.0,4.0
4,1048 sky lounge - wrigley rooftop,0.0,3.0


In [45]:
all_features_dataframe = features_dataframe.merge(yelp_dataset_grouped, how='left', on='Lowercased Name')

In [47]:
# Save to pickle
#all_features_dataframe.to_pickle('pickles/all_features_dataframe')

<font color='red'>

TO DO: Calculate Score

Idea:

We will compute the **Risk score** using a weighted sum of the features by their importance and use MinMax to normalize the score for each restaurant at the end

</font>

In [48]:
all_features_dataframe = pd.read_pickle('pickles/all_features_dataframe')